In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
import glob2
import json
import random
device="cuda"

/home/chitrank/miniconda3/envs/myenv_py3.8_another/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
embedder = SentenceTransformer('all-MiniLM-L6-v2').to(device)


In [3]:
embedder.requires_grad_(False)
# for k,v in embedder.named_parameters():
    # print(v.requires_grad)

    

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:

# eurlex_df_val = pd.read_json("./eurlex/eurlex_val_economy_visegrad_comecon.json")
# eurlex_df_train = pd.read_json("./eurlex/eurlex_train_economy_visegrad_comecon.json")
# eurlex_df_test = pd.read_json("./eurlex/eurlex_test_economy_visegrad_comecon.json")


In [5]:
train_rows=[]
for file in glob2.glob("eurlex/dataset/train/*.json"):
    x=json.load(open(file))
    # text=x["title"]+x["header"]
    train_rows.append(x)
eurlex_df_train = pd.DataFrame.from_records(train_rows)



In [6]:
val_rows=[]
for file in glob2.glob("eurlex/dataset/dev/*.json"):
    x=json.load(open(file))
    # text=x["title"]+x["header"]
    val_rows.append(x)
eurlex_df_val = pd.DataFrame.from_records(val_rows)


In [7]:
test_rows=[]
for file in glob2.glob("eurlex/dataset/test/*.json"):
    x=json.load(open(file))
    # text=x["title"]+x["header"]
    test_rows.append(x)
eurlex_df_test = pd.DataFrame.from_records(test_rows)


In [8]:
random.random()

0.23670903103331087

In [9]:
# eurlex_df_meta = pd.read_json("./eurlex/eurlex_concepts_metadata_economy_visegrad_comecon.json")
eurlex_df_meta = pd.read_json("./eurlex/eurovoc_en.json")
Eurlex_DAG = {}
index = 0
random.seed(41)
for concept in eurlex_df_meta.iloc[0]:
    if random.random()>0.1: continue
    if concept not in Eurlex_DAG:
        Eurlex_DAG[concept]=[]
    for parent in eurlex_df_meta.iloc[3][index]:
        if parent in Eurlex_DAG:
            Eurlex_DAG[parent].append(concept)
        else:
            Eurlex_DAG[parent] = [concept]
    index += 1

/tmp/ipykernel_638344/1153101309.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  eurlex_df_meta = pd.read_json("./eurlex/eurovoc_en.json")
/tmp/ipykernel_638344/1153101309.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  eurlex_df_meta = pd.read_json("./eurlex/eurovoc_en.json")
/tmp/ipykernel_638344/1153101309.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, mat

In [10]:
"5614" in Eurlex_DAG
len(Eurlex_DAG)

1174

In [11]:
label_to_global_index = {}
global_to_label_index = {}
index = 0
labels_set = set()
for parent in Eurlex_DAG.keys():
    if parent not in labels_set:
        global_to_label_index[index] = parent
        label_to_global_index[parent] = index
        index = index + 1
        labels_set.add(parent)
    for child in Eurlex_DAG[parent]:
        if child not in labels_set:
            global_to_label_index[index] = child
            label_to_global_index[child] = index
            index = index + 1
            labels_set.add(child)

In [12]:
eurlex_df_test

,celex_id,uri,type,concepts,title,header,recitals,main_body,attachments
0,32002R0515,http://publications.europa.eu/resource/cellar/...,Regulation,"[20, 3568, 4059]",Commission Regulation (EC) No 515/2002 of 21 M...,Commission Regulation (EC) No 515/2002\nof 21 ...,",\nHaving regard to the Treaty establishing th...",[For tenders notified from 15 to 21 March 2002...,"Done at Brussels, 21 March 2002.\nFor the Comm..."
1,32001R1448,http://publications.europa.eu/resource/cellar/...,Regulation,"[1086, 2734, 2965, 5138, 666, 976]",Council Regulation (EC) No 1448/2001 of 28 Jun...,Council Regulation (EC) No 1448/2001\nof 28 Ju...,",\nHaving regard to the Treaty establishing th...",[Regulation (EEC) No 3763/91 is hereby amended...,"Done at Luxembourg, 28 June 2001.\nFor the Cou..."
2,31985R1977,http://publications.europa.eu/resource/cellar/...,Regulation,"[1115, 1201, 1602, 2563, 863]",Council Regulation (EEC) No 1977/85 of 16 July...,COUNCIL REGULATION (EEC) No 1977/85\nof 16 Ju...,",\nHaving regard to the Treaty establishing th...",[The second subparagraph of Article 1 of Regul...,"Done at Brussels, 16 July 1985.\nFor the Counc..."
3,32005R1018,http://publications.europa.eu/resource/cellar/...,Regulation,"[1642, 2729]",Commission Regulation (EC) No 1018/2005 of 30 ...,1.7.2005 EN Official Journal of the European U...,",\nHaving regard to the Treaty establishing th...",[1. Notwithstanding Article 7(1) of Regulati...,"Done at Brussels, 30 June 2005.\nFor the Commi..."
4,32006R0841,http://publications.europa.eu/resource/cellar/...,Regulation,"[161, 1644, 4319, 4605]",Commission Regulation (EC) No 841/2006 of 7 J...,8.6.2006 EN Official Journal of the European U...,",\nHaving regard to the Treaty establishing th...",[In the case of import licence applications pr...,"Done at Brussels, 7 June 2006.\nFor the Commis..."
...,...,...,...,...,...,...,...,...,...
5995,32010D0006,http://publications.europa.eu/resource/cellar/...,Decision,"[2602, 3560, 4256, 4585, 5070, 56, 863, 935]","2010/6/EU, Euratom: Commission Decision of 22 ...",7.1.2010 EN Official Journal of the European U...,",\nHaving regard to the Treaty on the Function...",[For the purpose of calculating the VAT own re...,"Done at Brussels, 22 December 2009.\nFor the C..."
5996,31993R1317,http://publications.europa.eu/resource/cellar/...,Regulation,"[1102, 1565, 187, 3167]",COMMISSION REGULATION (EEC) No 1317/93 of 28 M...,COMMISSION REGULATION (EEC) No 1317/93 of 28 M...,",\nHaving regard to the Treaty establishing th...",[Regulation (EEC) No 1767/82 is hereby amended...,"Done at Brussels, 28 May 1993.\nFor the Commis..."
5997,32007R1325,http://publications.europa.eu/resource/cellar/...,Regulation,"[1085, 2282, 2437, 2879, 5228, 544, 598]",Commission Regulation (EC) No 1325/2007 of 12 ...,13.11.2007 EN Official Journal of the European...,",\nHaving regard to the Treaty establishing th...",[Quota exhaustion\nThe fishing quota allocated...,"Done at Brussels, 12 November 2007.\nFor the C..."
5998,32009R0215,http://publications.europa.eu/resource/cellar/...,Regulation,"[2883, 4381, 4654, 5410, 5751]",Commission Regulation (EC) No 215/2009 of 18 M...,19.3.2009 EN Official Journal of the European ...,",\nHaving regard to the Treaty establishing th...",[The goods described in column 1 of the table ...,"Done at Brussels, 18 March 2009.\nFor the Comm..."


In [13]:
A_directed=torch.zeros(len(labels_set),len(labels_set))
for t in range(len(labels_set)):
    if global_to_label_index[t] in Eurlex_DAG:
        for s in Eurlex_DAG[global_to_label_index[t]]:
            # A_directed[t][label_to_global_index[s]]=1
            A_directed[label_to_global_index[s]][t]=1
A_bidirec=A_directed+A_directed.T

In [14]:
A_bidirec.shape

torch.Size([1174, 1174])

In [15]:
labels = []
for i in range(len(labels_set)):
    labels.append(eurlex_df_meta[global_to_label_index[i]]['label'])
all_labels_encoded = embedder.encode(labels,convert_to_tensor=True)

In [16]:
eurlex_df_meta

,3474,1597,3363,4488,2316,5709,1107,2475,3294,278,...,4259,5826,3640,206,6145,452085,1449,4702,5661,2323
concept_id,3474,1597,3363,4488,2316,5709,1107,2475,3294,278,...,4259,5826,3640,206,6145,452085,1449,4702,5661,2323
label,international affairs,school legislation,union representative,data processing,barge,Lithuania,blue-veined cheese,construction policy,petrology,cost of borrowing,...,population statistics,promotion of the European idea,technological process,administrative cooperation,TRIMs,European fisheries fund,European Trade Union Institute,collective activities,ancient history,fishing season
alt_labels,[international politics],[],[trade union representative],"[automatic data processing, electronic data pr...",[canal boat],[Republic of Lithuania],"[Bresse blue, Danish blue, Roquefort, Stilton,...",[],[],[cost of credit],...,"[demographic indicator, demographic statistics...",[promotion of Europe],[],[],[trade-related investment measures],[EFF],"[ETUI-REHS, European Trade Union Institute for...",[],[ancient era],"[biological recovery, closed period for fishin..."
parents,[],[2467],[3374],[],[1036],"[122, 2200, 5283, 5774, 5781]",[1102],[],[1151],[2495],...,[1773],[1464],[4418],[2166],[1567],[1052],[5370],[4706],[1245],[5228]


In [17]:
def to_onehot(labels, n_categories, dtype=torch.float32): #Utilized a variant described in https://stackoverflow.com/questions/56123419/how-to-cover-a-label-list-under-the-multi-label-classification-context-into-one
    batch_size = len(labels)
    one_hot_labels = torch.zeros(size=(batch_size, n_categories), dtype=dtype)
    for i, label in enumerate(labels):
        label = torch.LongTensor(label)
        one_hot_labels[i] = one_hot_labels[i].scatter_(dim=0, index=label, value=1.)
    return one_hot_labels

In [18]:
class EurlexDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df):
        """
        """
        self.x_d=embedder.encode((df["title"]+" "+df["recitals"]).to_list(),convert_to_tensor=True)
        label_list = eurlex_df_train['concepts'].to_list()
        label_list = [[label_to_global_index[j] for j in i if j in label_to_global_index] for i in label_list]
        self.y_int = label_list
        self.y = to_onehot(label_list, len(global_to_label_index))
        

    def __len__(self):
        return len(self.x_d)

    def __getitem__(self, idx):
        return (self.x_d[idx], self.y[idx], self.y_int[idx])

In [19]:
to_onehot([[1,2],[0]],3)
# label_to_global_index
# A_bidirec

tensor([[0., 1., 1.],
        [1., 0., 0.]])

In [20]:
eurlex_train_dataset_gnn=EurlexDataset(eurlex_df_train)
eurlex_val_dataset_gnn=EurlexDataset(eurlex_df_val)
eurlex_test_dataset_gnn=EurlexDataset(eurlex_df_test)

In [21]:
torch.sum(eurlex_test_dataset_gnn.y)

tensor(47089.)

In [22]:
def eurlex_collate_fn(batch):
    x = torch.stack([i[0] for i in batch])
    y = torch.stack([i[1] for i in batch])
    y_int = [i[2] for i in batch]
    return x, y, y_int
    

In [23]:
eurlex_train_dl_gnn = torch.utils.data.DataLoader(eurlex_train_dataset_gnn, batch_size=32,shuffle=True, num_workers=0, collate_fn=eurlex_collate_fn)
eurlex_val_dl_gnn = torch.utils.data.DataLoader(eurlex_val_dataset_gnn, batch_size=64,shuffle=False, num_workers=0, collate_fn=eurlex_collate_fn)
eurlex_test_dl_gnn = torch.utils.data.DataLoader(eurlex_test_dataset_gnn, batch_size=64,shuffle=False, num_workers=0, collate_fn=eurlex_collate_fn)


In [24]:
def max_scores(val_scores,test_scores):
    max_epoch=0
    max_val_score=-1
    for epoch in range(len(val_scores)):
        if val_scores[epoch][0]>max_val_score:
            max_val_score=val_scores[epoch][0]
            max_epoch=epoch
    return max_epoch,val_scores[max_epoch],test_scores[max_epoch]

In [25]:
# embedder.encode(["ghbhb uhuygh"]).shape
hidden_dim=384

In [26]:
class GraphTransformer(torch.nn.Module):
    def __init__(self,num_convs):
        super(GraphTransformer, self).__init__()
        self.num_convs=num_convs
        self.node_representation = all_labels_encoded # Nl x D
        self.NL = all_labels_encoded.shape[0]
        # self.A=torch.Tensor(A_bidirec).to(device)
        self.A=torch.Tensor(A_directed).to(device)
        self.hidden_dim=hidden_dim
        self.A_til = (self.A/(1e-8+torch.sum(self.A,dim=-1,keepdim=True)))+torch.eye(self.NL).to(device)
        self.loss = torch.nn.BCEWithLogitsLoss(reduction = 'mean')
        self.lin1 = torch.nn.Linear(self.hidden_dim, 1)
        self.condensor = torch.nn.Linear(2*self.hidden_dim, self.hidden_dim) #Converts 2D -> D via a linear layer
        self.act=torch.nn.GELU()
        
    def GCNConv(self, x,option="cat,conv,cond"):
        B = x.shape[0]
        if option=="cat,conv,cond":
            return self.act(torch.matmul(self.A_til,x))
        else:
            return None

    def forward(self, x_d, y=None,option="cat,conv,cond"):
        """ 
        :param x_d = input document set (B x N x D)
        :return scores (B x N), loss (B)
        
        """
        B = x_d.shape[0]
        if option=="cat,conv,cond":
            x_prime = torch.tile(torch.unsqueeze(x_d, dim=1), dims=(1, self.NL, 1))# B x Nl x  D
            node_representation_m = torch.tile(torch.unsqueeze(self.node_representation, dim=0), dims=(B, 1, 1))    
            for i in range(self.num_convs): #Nodes with 5 from current node get information
                # print(x_prime.shape,node_representation_m.shape)
                x_prime=torch.cat((x_prime, node_representation_m), dim=-1)
                x_prime = self.GCNConv(x_prime,option)
                x_prime = self.condensor(x_prime)
                x_prime = self.act(x_prime)

            scores = torch.squeeze(self.lin1(x_prime), dim=-1) #Scores = (B x N)
            # print(scores.dtype,y.dtype)
            d_loss = self.loss(scores, y)
            return scores, d_loss


In [27]:
def eval_fn(model,dl):
    model.eval()
    k=10
    prec_at_1=0
    rec_at_k=0
    total_num=0
    correct_list=[]
    with torch.no_grad():
        for x_d,y,y_int in dl:
            scores,loss=model(x_d.to(device), y.to(device))
            topk = torch.topk(scores, k)[1].cpu().tolist()
            # print(topk,y_int)
            for pred,gt in zip(topk,y_int):
                # print(pred,gt)
                if len(gt)>0:
                    intersect_at_1=set.intersection(set(pred[:1]),set(gt))
                    intersect_at_k=set.intersection(set(pred),set(gt))
                    prec_at_1 += len(intersect_at_1)/1
                    rec_at_k += len(intersect_at_k)/len(gt)
                    total_num+=1
           
    return prec_at_1 /total_num,rec_at_k/total_num


In [31]:
eval_fn(model,eurlex_val_dl_gnn)
file="logs/eurlex_gnn4,5.txt"
print("New",file=open(file,"w"))
for num_convs in [4,5]:
    print("Num Convs",num_convs,"===================",file=open(file,"a"))
    model=GraphTransformer(num_convs).to(device)
    optim=torch.optim.AdamW(model.parameters(),lr=0.003)
    num_epochs = 100
    accuracies=[]
    val_scores=[]
    test_scores=[]
    for epoch in tqdm(range(num_epochs),desc="Epochs"):
    # for epoch in range(num_epochs):
        model.train()   
        for x_d, y, y_int in eurlex_train_dl_gnn:
            scores, loss = model(x_d.to(device), y.to(device))
            optim.zero_grad()
            loss.backward()
            optim.step()
        val_prec1,val_recall10 = eval_fn(model,eurlex_val_dl_gnn)
        test_prec1,test_recall10 = eval_fn(model,eurlex_test_dl_gnn)
        val_scores.append((val_prec1,val_recall10))
        test_scores.append((test_prec1,test_recall10))
        # print("Epoch ",epoch,"---",val_accuracy)        
    best_epoch,best_val_scores,best_test_scores=max_scores(val_scores,test_scores)
    print(best_epoch,best_val_scores,best_test_scores,file=open(file,"a"))

Epochs: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [4:15:03<00:00, 153.04s/it]


In [160]:
class BaselineModel(torch.nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
        self.d=hidden_dim
        self.act=torch.nn.GELU()
        self.ll1 = torch.nn.Linear(self.d, self.d)
        self.ll2 = torch.nn.Linear(self.d, self.d)
        self.ll3 = torch.nn.Linear(self.d, self.d)
        self.lin_class_lev2 = torch.nn.Linear(self.d, len(global_to_label_index))
        self.loss_fn=torch.nn.BCEWithLogitsLoss(reduction="mean")
    
    def forward(self,x_d, y=None):
        """
        :param
        x_d --> (B x D)
        x_l2 --> (B x D)
        
        """
        B= x_d.shape[0]
        loss=0
        x_d=self.act(self.ll1(x_d))
        x_d=self.act(self.ll2(x_d))
        x_d=self.act(self.ll3(x_d))
        scores=self.lin_class_lev2(x_d)  # returns B x num_lev1_child
        if y is not None:
            loss+=self.loss_fn(scores,y)
        
        return scores,loss
        
model=BaselineModel().to(device)
optim=torch.optim.AdamW(params=model.parameters(),lr=1e-3)

In [162]:
num_epochs = 100
accuracies=[]
for epoch in tqdm(range(num_epochs),desc="Epochs"):
    model.train()   
    for x_d, y, y_int in eurlex_train_dl_gnn:
        scores, loss = model(x_d.to(device), y.to(device))
        optim.zero_grad()
        loss.backward()
        optim.step()
    accuracy = eval_fn(model,eurlex_val_dl_gnn)
    accuracies.append(accuracy)
    # print("Epoch ",epoch,"---", accuracy)        
    val_scores.append((val_prec1,val_recall10))
    test_scores.append((test_prec1,test_recall10))
    # print("Epoch ",epoch,"---",val_accuracy)        
best_epoch,best_val_scores,best_test_scores=max_scores(val_scores,test_scores)
print(best_epoch,best_val_scores,best_test_scores)

Epochs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 19.42it/s]

Num Convs 5 ===================
1 (0.4864864864864865, 0.740990990990991) (0.5, 0.7467948717948718)
